Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = ""
COLLABORATORS = ""

---

<!--NOTEBOOK_HEADER-->
*This notebook contains material from [PyRosetta](https://RosettaCommons.github.io/PyRosetta);
content is available [on Github](https://github.com/RosettaCommons/PyRosetta.notebooks.git).*

<!--NAVIGATION-->
< [Side Chain Conformations and Dunbrack Energies](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.01-Side-Chain-Conformations-and-Dunbrack-Energies.ipynb) | [Contents](toc.ipynb) | [Index](index.ipynb) | [Design](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.03-Design.ipynb) ><p><a href="https://colab.research.google.com/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.02-Side-chain-packing.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>

# Side-Chain Packing
Keywords: PackerTask, standard_packer_task(), restrict_to_repacking(), temporarily_fix_everything(), temporarily_set_pack_residue(), PackRotamersMover(), SwitchResidueTypeSetMover(), MoveMap

## Monte Carlo

In [ ]:
# Notebook setup
import sys
if 'google.colab' in sys.modules:
    !pip install pyrosettacolabsetup
    import pyrosettacolabsetup
    pyrosettacolabsetup.setup()
    print ("Notebook is set for PyRosetta use in Colab.  Have fun!")

**Make sure you are in the directory with the pdb files:**

`cd google_drive/My\ Drive/student-notebooks/`

In [ ]:
# From previous section:
from pyrosetta import *
from pyrosetta.teaching import *
pyrosetta.init()
pose = pose_from_pdb("inputs/1YY8.clean.pdb")
start_pose = Pose()
start_pose.assign(pose)
scorefxn = get_fa_scorefxn()

Side-chain packing can be done in a Monte Carlo search routine that iteratively swaps rotamers of a random residue and tests each move using the Metropolis criterion. Rosetta has such a routine pre-packaged as a `Mover` that carries out a simulated annealing search each time it is applied. The specific scope of the packing is specified in a `PackerTask` object, which is similar to a `MoveMap` in that it specifies degrees of freedom. We can specify via commands or from an input file our settings for a `PackerTask`.

Create a `PackerTask` as follows. This will set the task to allow packing only of residue 49:

```
task_pack = standard_packer_task(pose)
task_pack.restrict_to_repacking()
task_pack.temporarily_fix_everything()
task_pack.temporarily_set_pack_residue(49, True)
```

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

The default task allows any amino acid residue to be swapped in for another; that is, it would simulate a protein variant as a result of mutation. This would be useful for protein design but not for side-chain packing. `restrict_to_repacking()` only allows rotamers from the current residue at that position to be used.

We can confirm our settings using (note how only one amino acid is allowed at each position):
```
print(task_pack)
```

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

We now can construct a `PackRotamersMover`:

```
pack_mover = PackRotamersMover(scorefxn, task_pack)
```

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Apply the `PackMover` above to your pose with the `.apply()` method.

```
pack_mover.apply(pose)
```

__Question:__ Now what are the χ angles of K49? Which rotamer is this? What is the Dunbrack energy?

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

__Question:__ What is the new total energy of K49? Why did Rosetta pick this rotamer? Answer this in terms of the components of the score function and in terms of the residues with which K49 interacts.

## Packing for Refinement


Side-chain packing can be used when converting a pose from centroid to full-atom mode, and it is used extensively in full-atom refinement calculations. Let’s examine how packing improves scores.

Create a centroid-representation model for RecA protein domain 2 (PDB ID: 2REB) using the `SwitchResidueTypeSetMover`. Save that centroid “decoy” so that we can compare several basic refinement steps.

```
cen_ras = pose_from_file("6Q21_A.pdb")
switch = SwitchResidueTypeSetMover("centroid")
switch.apply(cen_ras)
```

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Load another `ras` and keep it in full-atom representation. Save this starting configuration for future use. Score the pose with the standard centroid score function.

__Question:__ Why is the score so high?

```
ras = pose_from_file("6Q21_A.pdb")
start_ras = Pose()
start_ras.assign(ras)
scorefxn(ras)
```

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Create a default `PackRotamersMover` with a `PackerTask` that allows all residues to vary χ angles. Create a test pose from your start pose and pack the side chains.

__Question:__ What is the new pose score?

```
test_ras = Pose()
test_ras.assign(start_ras)

task_pack = standard_packer_task(test_ras)
task_pack.restrict_to_repacking()
task_pack.temporarily_fix_everything()
task_pack.temporarily_set_pack_residue(49, True)

pack_mover = PackRotamersMover(scorefxn, task_pack)
pack_mover.apply(test_ras)
```

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Reset the test pose to the start configuration. Create a `MoveMap` that allows χ angles but not φ/ψ/ω angles to vary. Confirm the `MoveMap` by printing it. Create a `MinMover` using the Davidson-Fletcher-Powell minimization scheme by applying the method `min_type("dfpmin")` to your mover. Apply the `MinMover` and rescore the pose.

__Question:__ How does this energy compare?

```
test_ras.assign(start_ras)
mm = MoveMap()
mm.set_chi(True)
mm.set_bb(False)
print(mm)

min_mover = MinMover()
min_mover.set_movemap(mm)
min_mover.score_function(scorefxn)
min_mover.min_type("dfpmin")
print(min_mover)

print(scorefxn(test_ras))
min_mover.apply(test_ras)
print(scorefxn(test_ras))
```

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Again, reset the test pose to the starting configuration. Apply the packer and then minimize on the χ angles.

__Question:__ Now what is the final score?

```
test_ras.assign(start_ras)
print(scorefxn(ras))
pack_mover.apply(ras)
print(scorefxn(ras))
min_mover.apply(ras)
print(scorefxn(ras))
```

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

For fun, you might examine the individual residue energies to find the residues most responsible for the score changes. Typically, a small number of residues may make clashes that can be resolved using the χ angle minimization, which allows off-rotamer side-chain conformations.

<!--NAVIGATION-->
< [Side Chain Conformations and Dunbrack Energies](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.01-Side-Chain-Conformations-and-Dunbrack-Energies.ipynb) | [Contents](toc.ipynb) | [Index](index.ipynb) | [Design](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.03-Design.ipynb) ><p><a href="https://colab.research.google.com/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.02-Side-chain-packing.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>